In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../Dataset'))
sys.path.append(os.path.abspath('..'))

import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from func_preprocesamiento import preprocesar, preprocesar_binarios, procesar_antiguedad, preprocesar_categoricos

In [2]:
dev_df = pd.read_csv('../Dataset/raw/dev.csv')


train_dataset, val_dataset = train_test_split(dev_df, train_size=0.2, random_state=42)
X_train = train_dataset.drop(columns=['precio_pesos_constantes'])
y_train = train_dataset['precio_pesos_constantes']

/tmp/ipykernel_11199/3635508779.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_df = pd.read_csv('../Dataset/raw/dev.csv')


In [ ]:
df = preprocesar(train_dataset)
direccion = '../Dataset/Procesado/dev_procesado.csv'
df.to_csv(direccion, index=False)

Columna a predecir: ITE_ADD_NEIGHBORHOOD_NAME_encoded
Vector real: [0.01021463 0.01131207 0.0255999  ... 0.00835743 0.01306376 0.01266277]
Vector predicho: [0.01201781 0.03463099 0.03301163 ... 0.00996412 0.01306376 0.00806745]
RMSE: 0.009319067977126427
Columna a predecir: SalonDeUsosMul
Columna a predecir: AireAC
Columna a predecir: Estacionamiento
Columna a predecir: Seguridad
Columna a predecir: AreaParrillas
Columna a predecir: CanchaTennis
Columna a predecir: AreaCine
Columna a predecir: Gimnasio
Columna a predecir: Amoblado
Columnas independientes: ['Dormitorios', 'Banos', 'Ambientes', 'Cocheras', 'STotalM2', 'SConstrM2', 'LONGITUDE', 'LATITUDE', 'ITE_ADD_CITY_NAME_encoded', 'ITE_ADD_STATE_NAME_encoded', 'ITE_ADD_NEIGHBORHOOD_NAME_encoded', 'SalonDeUsosMul', 'AireAC', 'Estacionamiento', 'Seguridad', 'AreaParrillas', 'CanchaTennis', 'AreaCine', 'Gimnasio', 'Amoblado', 'Cisterna', 'BusinessCenter', 'Laundry', 'Jacuzzi', 'Chimenea', 'Ascensor', 'EstacionamientoVisitas', 'Lobby', 'L

In [4]:
columna = ['ITE_ADD_CITY_NAME', 'ITE_ADD_STATE_NAME', 'ITE_ADD_NEIGHBORHOOD_NAME', 'ITE_TIPO_PROD', 'TIPOPROPIEDAD'] #SACARLAS PORQUE SON CATEGORICAS
direccion = '../Dataset/Procesado/dev_procesado_numerico.csv'
df = df.drop(columns=columna)
df.to_csv(direccion, index=False)

In [9]:
import pandas as pd

# Cargar el DataFrame
dev_procesado = pd.read_csv('../Dataset/Procesado/dev_procesado_numerico.csv')

# Verificar si las columnas existen
if 'STotalM2' in dev_procesado.columns and 'SConstrM2' in dev_procesado.columns:
    # Filtrar los valores donde STotalM2 es menor que SCubiertaM2
    filtro = dev_procesado['STotalM2'] < dev_procesado['SConstrM2']
    valores_menores = dev_procesado[filtro]
    
    # Imprimir los valores
    print("Valores donde STotalM2 es menor que SConstrM2:")
    print(valores_menores[['STotalM2', 'SConstrM2']])

Valores donde STotalM2 es menor que SConstrM2:
       STotalM2  SConstrM2
100         7.0       43.0
248        16.0      165.0
404        50.0       55.0
442        65.0    70000.0
664        50.0       55.0
...         ...        ...
46988      28.0       29.0
47042      41.0       58.0
47103      50.0       55.0
47241      40.0       41.0
47330      40.0       65.0

[447 rows x 2 columns]
